<a href="https://colab.research.google.com/github/chin0308/Research_Papers/blob/main/HUD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python-headless numpy

In [ ]:
from google.colab import files

uploaded = files.upload()  # This will prompt you to select a file from your computer.


Saving input_video.mp4 to input_video (1).mp4


In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow  # for optional display of images in Colab

from google.colab import files


# Get the name of the uploaded file
video_filename = list(uploaded.keys())[0]

# Define the video path
video_path = video_filename
cap = cv2.VideoCapture(video_path)

# ... (rest of your video processing code remains the same) ...

# Download the output video
#files.download(output_path)

# Define the video path (make sure the uploaded file is named exactly 'input_video.mp4')
video_path = video_filename # Changed 'input_video.mp4' to video_filename
cap = cv2.VideoCapture(video_path)

# Get video properties
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Set up the output video writer
output_path = 'output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale for processing
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Detect edges (for lanes)
    edges = cv2.Canny(blurred, 100, 200)
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, 50, minLineLength=50, maxLineGap=10)

    # Draw lane lines
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Detect cars (simple contour method)
    _, thresh = cv2.threshold(blurred, 100, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    overtake_safe = False
    for cnt in contours:
        if cv2.contourArea(cnt) > 500:
            x, y, w, h = cv2.boundingRect(cnt)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
            distance = 1000 / w
            if distance > 20 and lines is not None:
                overtake_safe = True

    # Add overtaking guidance (HUD-like overlay)
    if overtake_safe:
        cv2.putText(frame, "Clear to overtake", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    else:
        cv2.putText(frame, "Wait", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Write the frame to the output video
    out.write(frame)

    # Optional: Display the first frame for verification (uncomment if needed)
    # if frame_count == 0:
    #     cv2_imshow(frame)  # Use cv2_imshow from google.colab.patches
    # frame_count += 1

# Clean up
cap.release()
out.release()
cv2.destroyAllWindows()

print("Processing complete. Output video saved as", output_path)


# Download the output video # Moved this line to the end after video processing is complete.
files.download(output_path)


Processing complete. Output video saved as output_video.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>